# KOL Anomaly Detection
http://scikit-learn.org/stable/modules/outlier_detection.html#id1

In [1]:
# Import libraries

import teradata as td
import pandas as pd
import numpy as np
from getpass import getpass, getuser
# import matplotlib.pyplot as plt

In [2]:
# Log in to Teradata

udaExec = td.UdaExec(appName="Pfizer_KOL_CMS_App", version="1.0",
        logConsole=False)

uname = getuser()
print(uname)
pword = getpass("Teradata Password: ")
 
session = udaExec.connect(method="odbc", system="amrndhp3",
        username=uname, password=pword);

WEITEM
Teradata Password: ········


In [3]:
# Get CMS data (takes about 30 seconds)

listed_data = []
data = session.execute("SELECT * FROM HVAD_TEAM_T.mw_cms_kol_all_2k17")
for row in data:
    listed_data.append(row[0:])

In [4]:
# Get column names

col_names = []
col_names_ter = session.execute("SELECT ColumnName FROM dbc.ColumnsV WHERE DatabaseName='HVAD_TEAM_T' AND TableName='mw_cms_kol_all_2k17' ORDER BY ColumnId")
for item in col_names_ter:
    col_names.append(item[0])
col_names

['record_id',
 'phys_full_name',
 'assoc_type',
 'product',
 'payment_date',
 'amount',
 'num_payments',
 'payment_nature',
 'Payment_Year',
 'submitting_company',
 'paying_company',
 'consolidating_company']

In [5]:
# Initialize dataframe

df = pd.DataFrame(data=listed_data, columns = col_names)
df.head()

,record_id,phys_full_name,assoc_type,product,payment_date,amount,num_payments,payment_nature,Payment_Year,submitting_company,paying_company,consolidating_company
0,389957222,"Soloman, Nehad",recipient,PROLIA,2016-10-06,123.19,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.
1,31093345,"Strand, Vibeke",recipient,None,2013-11-06,65.07,1,Travel and Lodging,2013,"AbbVie, Inc.","AbbVie, Inc.","AbbVie, Inc."
2,309188896,"Aelion, Jacob",recipient,HUMIRA,2015-02-25,12.95,1,Food and Beverage,2015,"AbbVie, Inc.","AbbVie, Inc.","AbbVie, Inc."
3,359329330,"Lambert, Rhonda E",recipient,KRYSTEXXA,2016-08-12,43.14,1,Food and Beverage,2016,Horizon Pharma plc,Horizon Pharma plc,Horizon Pharma plc
4,388751724,"Huffstutter, Joseph E",recipient,ENBREL,2016-11-11,12.40,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.


In [8]:
# Improve formatting for numerical data

df['Payment_Year'] = df['Payment_Year'].apply(int)
df['amount'] = df['amount'].apply(float)
df['num_payments'] = df['num_payments'].dropna()
df['num_payments'] = df['num_payments'].apply(float)

In [9]:
# Create unique int labels for categorical data so the Unsupervised Learning algos can process the dataframe

for colname in ['phys_full_name','assoc_type','payment_date','product','payment_nature','consolidating_company']:
    dkeys = df[colname].unique()
    dvalues = list(range(len(df[colname].unique())))
    dmapping = dict(zip(dkeys, dvalues))
    df = df.replace({colname: dmapping})

In [15]:
# Exclude irrelevant columns and incomplete rows

df = df.drop(['submitting_company', 'paying_company'], axis=1)
df = df.dropna()
df.head()

,record_id,phys_full_name,assoc_type,product,payment_date,amount,num_payments,payment_nature,Payment_Year,consolidating_company
0,389957222,0,0,0,0,123.19,1.0,0,2016,0
1,31093345,1,0,1,1,65.07,1.0,1,2013,1
2,309188896,2,0,2,2,12.95,1.0,0,2015,1
3,359329330,3,0,3,3,43.14,1.0,0,2016,2
4,388751724,4,0,4,4,12.40,1.0,0,2016,0


In [ ]:
# Separate categorical data and numerical data just in case we want to analyze either individually

# categorical_data = df[['record_id','phys_full_name','assoc_type','payment_date','product','payment_nature','submitting_company',
#                        'paying_company','consolidating_company']]
# numerical_data = df.drop(['record_id','phys_full_name','assoc_type','payment_date','product','payment_nature',
#                           'consolidating_company','submitting_company','paying_company'], axis=1)
# numerical_data.head()

In [ ]:
# Non ML outlier-detection:
# Keep only the outliers (outside +3 to -3 standard deviations)

# outliers = numerical_data[(np.abs(numerical_data-numerical_data.mean())>(3*numerical_data.std()))].dropna(how='all')
outliers = df[(np.abs(df-df.mean())>(3*df.std()))].dropna(how='all')

In [27]:
# Make a K-Neighbors Graph. Not sure what this does yet.
# http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.kneighbors_graph

k=3
# X = numerical_data
nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(df)
distances, indices = nbrs.kneighbors(df)
nbrs.kneighbors_graph(df)

<94886x94886 sparse matrix of type '<class 'numpy.float64'>'
	with 284658 stored elements in Compressed Sparse Row format>

In [22]:
# Isolation Forest Scoring
from sklearn.ensemble import IsolationForest

Forest = IsolationForest()
Forest.fit(df)
df['IForest Score'] = Forest.decision_function(df)

In [19]:
# Elliptic Envelope Scoring
from sklearn.covariance import EllipticEnvelope

Envelope = EllipticEnvelope()
Envelope.fit(df)
df['Elliptical Score'] = Envelope.decision_function(df)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:622: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [ ]:
# Local Outlier Factor Labeling
from sklearn.neighbors import LocalOutlierFactor

LOF = LocalOutlierFactor()
LOF.fit(df)
df['LOF Labels'] = LOF.fit_predict(df)

In [ ]:
# x = numerical_data['Payment_Year']
# y = numerical_data['amount']

# plt.figure()
# plt.scatter(x, y)
# plt.show()

# plt.figure()
# plt.scatter(numerical_data['Payment_Year'], numerical_data['num_payments'])
# plt.show()

# plt.figure()
# plt.scatter(numerical_data['num_payments'], numerical_data['amount'])
# plt.show()

In [26]:
df.head()

,record_id,phys_full_name,assoc_type,product,payment_date,amount,num_payments,payment_nature,Payment_Year,consolidating_company,IForest Score,Elliptical Score,LOF Labels
0,389957222,0,0,0,0,123.19,1.0,0,2016,0,0.024290,0.807282,1
1,31093345,1,0,1,1,65.07,1.0,1,2013,1,-0.023504,-0.417888,1
2,309188896,2,0,2,2,12.95,1.0,0,2015,1,0.089157,1.371532,1
3,359329330,3,0,3,3,43.14,1.0,0,2016,2,-0.018008,1.042721,-1
4,388751724,4,0,4,4,12.40,1.0,0,2016,0,0.056176,0.800109,1
